In [1]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()), 'research/qd/diffusion_models')
os.chdir(project_root)
%pwd # should be PPGA root dir

'/home/shashank/research/qd/diffusion_models'

In [2]:
import torch
import numpy as np
import os

from pathlib import Path
from torch.optim import Adam
from autoencoders.transformer_autoencoder import AutoEncoder
from autoencoders.policy_hyperautoencoder import HyperAutoEncoder
from autoencoders.conv_autoencoder import AutoEncoder as ConvVAE
from dataset.mnist_fashion_dataset import dataloader
from losses.loss_functions import normal_kl
from losses.contperceptual import LPIPSWithDiscriminator

from dataset.policy_dataset import e_data_loader_train, actor_cfg
import torch.nn.functional as F



/home/shashank/miniconda3/envs/qd/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset fashion_mnist (/home/shashank/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)
100%|██████████| 791/791 [00:01<00:00, 439.04it/s]


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = AutoEncoder(emb_channels=8, z_channels=4)
# model = ConvVAE()
model = HyperAutoEncoder(actor_cfg, emb_channels=8, z_channels=4)
model.to(device)

optimizer = Adam(model.parameters(), lr=1e-4)

mse_loss_func = torch.nn.MSELoss()
kl_loss_coef = 1e-6

model_checkpoint_folder = Path('./checkpoints2')
model_checkpoint_folder.mkdir(exist_ok=True)

disc_start = 50001
kl_weight = 1e-6
disc_weight = 0.5
# loss_func = LPIPSWithDiscriminator(disc_start, kl_weight=kl_weight, disc_weight=disc_weight)
# optimizer2 = Adam(loss_func.discriminator.parameters(), lr=1e-3)


epochs = 30
global_step = 0
for epoch in range(epochs):
    print(f'{epoch=}')
    print(f'{global_step=}')

    epoch_loss = 0

    for step, batch in enumerate(e_data_loader_train):
        optimizer.zero_grad()

        # batch = batch['pixel_values'].to(device)
        batch = batch[0]

        # img_out, posterior = model(batch)
        img_out = model(batch)
        # loss = loss_func(batch, img_out, posterior, global_step, 0)
        # loss += loss_func(batch, img_out, posterior, global_step, 1)
        pred_weights_dict = {}
        for out_ in img_out:
            for name, param in out_.named_parameters():
                if name not in pred_weights_dict:
                    pred_weights_dict[name] = []
                pred_weights_dict[name].append(param)
                

        # loss is MSE loss
        mse_loss = 0
        for key in pred_weights_dict:
            mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), batch[key])

        # loss = mse_loss_func(batch, img_out) + kl_loss_coef * posterior.kl().mean()
        # loss = mse_loss + kl_loss_coef * posterior.kl().mean()
        loss = mse_loss 

        loss.backward()
        # if step % 100 == 0:
            # print(f'Loss: {loss.item()}, MSE: {mse_loss.item()}, KL: {posterior.kl().mean().item()}')
            # print(f'Loss: {loss.item()}, MSE: {mse_loss.item()}')
            # print(f'grad norm: {grad_norm(model)}')
        epoch_loss += loss.item()
        optimizer.step()

    global_step += step
    print(f'Epoch: {epoch}, Loss: {loss.item()}, MSE: {epoch_loss/len(e_data_loader_train)}')
print('Saving final model checkpoint...')
torch.save(model.state_dict(), os.path.join(str(model_checkpoint_folder), 'autoencoder.pt'))



KeyError: 'module name can\'t contain ".", got: actor_mean.0.weight'

In [15]:
sampled_x,y = e_data_loader_train.__iter__().__next__()
img_out = model(sampled_x)
# loss = loss_func(batch, img_out, posterior, global_step, 0)
# loss += loss_func(batch, img_out, posterior, global_step, 1)
pred_weights_dict = {}
for out_ in img_out:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), sampled_x[key])

mse_loss

tensor(0.0060, device='cuda:0', grad_fn=<AddBackward0>)

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model2 = HyperAutoEncoder(actor_cfg, emb_channels=8, z_channels=4)
model2.to(device)

In [17]:
sampled_x,y = e_data_loader_train.__iter__().__next__()
img_out = model2(sampled_x)
# loss = loss_func(batch, img_out, posterior, global_step, 0)
# loss += loss_func(batch, img_out, posterior, global_step, 1)
pred_weights_dict = {}
for out_ in img_out:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), sampled_x[key])

mse_loss

tensor(0.1967, device='cuda:0', grad_fn=<AddBackward0>)

In [18]:
model_cp = f"./{model_checkpoint_folder}/autoencoder.pt"
model2.load_state_dict(torch.load(model_cp))

<All keys matched successfully>

In [19]:
sampled_x,y = e_data_loader_train.__iter__().__next__()
img_out = model2(sampled_x)
# loss = loss_func(batch, img_out, posterior, global_step, 0)
# loss += loss_func(batch, img_out, posterior, global_step, 1)
pred_weights_dict = {}
for out_ in img_out:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), sampled_x[key])

mse_loss

tensor(0.0993, device='cuda:0', grad_fn=<AddBackward0>)

In [55]:
model2 = HyperAutoEncoder(actor_cfg, emb_channels=8, z_channels=4)
# model2.decoder.load_state_dict(model.decoder.state_dict())
# model2.encoder.load_state_dict(model.encoder.state_dict())
model2.encoder = model.encoder
model2.decoder = model.decoder
model2.to(device)

In [54]:
sampled_x,y = e_data_loader_train.__iter__().__next__()
img_out = model2(sampled_x)
# loss = loss_func(batch, img_out, posterior, global_step, 0)
# loss += loss_func(batch, img_out, posterior, global_step, 1)
pred_weights_dict = {}
for out_ in img_out:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), sampled_x[key])

mse_loss

tensor(0.0069, device='cuda:0', grad_fn=<AddBackward0>)